In [73]:
from array import array
import reprlib
import math
import numbers
import functools
import operator

# 完整版的vector
class Vector():
    typecode = 'd'
    shortcut_name = 'xyzt'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return bytes([ord(self.typecode)]) + bytes(self._components)

    def __eq__(self, other):
        # return tuple(self) == tuple(other)该方法内存消耗较大
        return len(other) == len(self) and all(a == b for a,  b in zip(self._components,other))

    def __abs__(self):
        return math.sqrt(sum(x ** 2 for x in self))

    def __bool__(self):
        return bool(abs(self))

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be intergers'
            raise TypeError(msg.format(cls=cls))

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_name.find(name)
            if 0<= pos <len(self._components):
                return self._components[pos]

        msg = '{cls.__name__} object has no attribute {name}'
        raise AttributeError(msg.format(cls=cls, name= name))

    def __setattr__(self, key, value):
        cls = type(self)
        if len(key)==1:
            if key in cls.shortcut_name:
                error = 'readonly attribute {attr_name!r}'
            elif key.islower():
                error = 'Can not set "a" to "z" in {cls_name!r}'
            else:
                error=''
            if error:
                msg = error.format(cls_name=cls.__name__,attr_name=key)
                raise AttributeError(msg)

        # 否则就调用默认情况
        super().__setattr__(key, value)
        
    def __hash__(self):
        hashes = (hash(x) for x in self._components)
        return functools.reduce(operator.xor, hashes, 0)
    

In [74]:
v1 = Vector([3.2, 2.4, 5.6, 6.7])

In [75]:
repr(v1)

'Vector([3.2, 2.4, 5.6, 6.7])'

In [76]:
bytes(v1)

b'd\x9a\x99\x99\x99\x99\x99\t@333333\x03@ffffff\x16@\xcd\xcc\xcc\xcc\xcc\xcc\x1a@'

In [77]:
for x in v1:
    print(x)

3.2
2.4
5.6
6.7


In [78]:
# 协议和鸭子类型

In [79]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck():
    rank = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds club hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank ,suit) for suit in self.suits
                                        for rank in self.rank]
    
    def __getitem__(self, item):
        return self._cards[item]
    
    def __len__(self):
        return len(self._cards)

In [80]:
# 切片原理

In [81]:
class MYSeq():
    def __getitem__(self, item):
        return item

In [82]:
s = MYSeq()

In [83]:
s[1:4:2,3:9]

(slice(1, 4, 2), slice(3, 9, None))

In [84]:
dir(slice)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'indices',
 'start',
 'step',
 'stop']

In [85]:
help(slice.indices)

Help on method_descriptor:

indices(...)
    S.indices(len) -> (start, stop, stride)
    
    Assuming a sequence of length len, calculate the start and stop
    indices, and the stride length of the extended slice described by
    S. Out of bounds indices are clipped in a manner consistent with the
    handling of normal slices.



In [86]:
slice(None, 10, 2).indices(5) #arrray[:10:2] but len(array)==5 so result: array[:5:2]

(0, 5, 2)

In [87]:
v1 = Vector(range(10))

In [88]:
repr(v1)

'Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])'

In [89]:
v1[0:3:2]

Vector([0.0, 2.0])

In [90]:
v1.x = 10 #当v.x被赋值后便有了x属性，但这里我们要实现的是改变v1[0]的数

AttributeError: readonly attribute 'x'

In [91]:
print(v1.x, v1[0])

0.0 0.0


In [92]:
# 所以注意定义__getattr__方法后也要定义__setattr__方法

In [93]:
#Test the final vector

In [94]:
Vector([3.2,5.2])

Vector([3.2, 5.2])

In [95]:
Vector(range(10))

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

In [96]:
v1 = Vector([3,4])

In [97]:
x, y = v1
x,y

(3.0, 4.0)

In [98]:
bytes(v1)

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [100]:
v1[:3]

Vector([3.0, 4.0])